<a href="https://colab.research.google.com/github/tubagokhan/GrinCH/blob/main/Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install py-rouge==1.1
!pip install sentence_transformers
!pip install evaluate
!pip install bert_score

In [2]:
import rouge
from sentence_transformers import SentenceTransformer
import numpy as np
from nltk import sent_tokenize

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## ROUGE

In [3]:
def prepare_results(m, p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(m, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)

def rougeEvaluation(all_hypothesis, all_references):

    for aggregator in ['Avg']:
        print('Evaluation with {}'.format(aggregator))
        apply_avg = aggregator == 'Avg'
        apply_best = aggregator == 'Best'

        evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                               max_n=4,
                               limit_length=False,
                               length_limit=1000,
                               length_limit_type='words',
                               apply_avg=apply_avg,
                               apply_best=apply_best,
                               alpha=0.2, # Default F1_score
                               weight_factor=1.2,
                               stemming=True)

        scores = evaluator.get_scores(all_hypothesis, all_references)

        for metric, results in sorted(scores.items(), key=lambda x: x[0]):
            if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
                for hypothesis_id, results_per_ref in enumerate(results):
                    nb_references = len(results_per_ref['p'])
                    for reference_id in range(nb_references):
                        print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
                        print('\t' + prepare_results(metric,results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
                print()
            else:
                print(prepare_results(metric, results['p'], results['r'], results['f']))
        print()

## RESCORE

In [4]:

def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

def redundancyMeasure(corpus):
    sentences=sent_tokenize(corpus)
    length=len(sentences)

    #print("Sentence Count:"+ str(len(sentences)))

    if(len(sentences)<2):
      return 0

    model = SentenceTransformer('bert-base-nli-mean-tokens')
    sentence_embeddings = model.encode(sentences)

    sentenceGraph =np.zeros((length, length))
    for x in range(length):
        for y in range(length):
            if x>y:
                similarity= cosine(sentence_embeddings[x],sentence_embeddings[y])
                sentenceGraph[x][y]=abs(similarity)
    #print(sentenceGraph)
    SumElement=(np.concatenate(sentenceGraph).sum())
    divide=(length-1)*length/2

    similarityScore= SumElement/divide

    #similarityScore2 =np.mean(sentenceGraph, dtype = np.float64)
    #print(similarityScore2)

    return (round(similarityScore,2))

def ReSCORE(corpusList):
  listOfSimilarityDoc=[]
  for doc in corpusList:
    docSimilarity=redundancyMeasure(doc)
    listOfSimilarityDoc.append(docSimilarity)
    #print(len(listOfSimilarityDoc))

  ReScore=round(sum(listOfSimilarityDoc)/len(listOfSimilarityDoc),2)
  return ReScore

## READ FILES

In [5]:
# Save files

import json
import os

# Define the file path
file_path = "/content/GraphInClusterPubMedTextRanksavedDocumentsandSummaries.json"  # Replace with the desired file path

all_summary=[]
all_goldstandart=[]

# Check if the file already exists
if os.path.exists(file_path):
    # Read the existing data from the JSON file
    with open(file_path, "r") as file:
        existing_data = json.load(file)

    # Combine the existing data with the new data (lists)
    all_summary = existing_data.get("all_summary", [])
    all_goldstandart = existing_data.get("all_goldstandart", [])


## EVALUATION

In [6]:
len(all_summary)

500

In [7]:
len(all_goldstandart)

500

In [8]:
rougeEvaluation(all_summary, all_goldstandart)

Evaluation with Avg
	rouge-1:	P: 36.67	R: 48.36	F1: 43.43
	rouge-2:	P: 12.57	R: 15.76	F1: 14.30
	rouge-3:	P:  6.48	R:  7.53	F1:  6.92
	rouge-4:	P:  4.23	R:  4.56	F1:  4.24
	rouge-l:	P: 34.33	R: 42.97	F1: 39.62
	rouge-w:	P: 16.42	R: 11.09	F1: 11.43



In [9]:
# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [10]:
from evaluate import load
bertscore = load("bertscore")
results = bertscore.compute(predictions=all_summary, references=all_goldstandart, lang="en", model_type="distilbert-base-uncased")
print(results)

{'precision': [0.8055368065834045, 0.7985310554504395, 0.761563777923584, 0.7727776169776917, 0.8107786178588867, 0.7647646069526672, 0.8209121227264404, 0.7852374911308289, 0.8045998811721802, 0.7747252583503723, 0.7787547707557678, 0.0, 0.8342681527137756, 0.760868489742279, 0.8966976404190063, 0.8008041381835938, 0.800202488899231, 0.8282234072685242, 0.8017926812171936, 0.8053169846534729, 0.7394133806228638, 0.792189359664917, 0.0, 0.7846925854682922, 0.7134657502174377, 0.8018949627876282, 0.7844705581665039, 0.7470893859863281, 0.7913129925727844, 0.7963830828666687, 0.7770141363143921, 0.7919992208480835, 0.7986537218093872, 0.7482227683067322, 0.7490156888961792, 0.7589308023452759, 0.7508138418197632, 0.8070206642150879, 0.7974945306777954, 0.8316938281059265, 0.8085888624191284, 0.8002365827560425, 0.8151413202285767, 0.804214596748352, 0.7806628346443176, 0.80863356590271, 0.7954121828079224, 0.7679427862167358, 0.7424330711364746, 0.8081576824188232, 0.0, 0.742301821708679

In [11]:
# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [12]:
print(ReSCORE(all_summary))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [13]:
print(ReSCORE(all_goldstandart))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [14]:
# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')